# Faster than light travel

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lookbusy1344/Relativity/blob/main/Python/FTL%20Travel%20animation.ipynb)

This is an animated version of the Minkowski diagram for time travelling FTL. A fuller notebook with more references is here:

https://github.com/lookbusy1344/Relativity/blob/main/Python/FTL%20Travel.ipynb

## How does FTL travel violate causality?

FTL would mean travel into the past is possible.

This stems from the relativity of simultaneity in Special Relativity. Events that are simultaneous in one frame of reference may not be simultaneous in another frame moving relative to the first. If FTL travel were possible, it would allow information or objects to travel back in time in some frames of reference, leading to causality violations. Here is a detailed example:

1. We use our warp drive to travel 4.2 light years to Proxima, instantaneously in our frame of reference.
2. We now accelerate to 0.99c.
3. In our new boosted frame, we warp back to Earth instantaneously.
4. We arrive back on Earth years before we left!

These paradoxes are explored in the `ftl_lib.py` library.

Lets build a Minkowski diagram to visualise the time travelling mission:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import time

# Constants
distance = 4.2  # light years to Proxima
v_final = 0.99  # final boost velocity as fraction of c
theta_final = np.arctan(v_final)  # angle for boosted simultaneity
time_shift_final = -v_final * distance  # simultaneity shift ≈ -4.158 years

# Start benchmark
start_time = time.perf_counter()

# Create figure for animation
fig, ax = plt.subplots(figsize=(12, 10))


def init():
    ax.clear()
    ax.set_xlabel("Distance (light years)", fontsize=13)
    ax.set_ylabel("Time (years)", fontsize=13)
    ax.set_title(
        "FTL Travel and Causality Violation\n(Animating Lorentz Boost)",
        fontsize=16,
        fontweight="bold",
    )
    ax.grid(True, alpha=0.3)
    ax.set_aspect("equal")
    ax.axhline(y=0, color="k", linewidth=0.5)
    ax.axvline(x=0, color="k", linewidth=0.5)
    ax.set_xlim(-2.2, 5)
    ax.set_ylim(-5.5, 3)
    return []


def animate(frame):
    ax.clear()

    # Calculate current velocity (animate from 0 to v_final)
    progress = frame / 50.0  # 0 to 1 for 51 frames (0-50)
    v_current = progress * v_final
    theta_current = np.arctan(v_current)
    time_shift_current = -v_current * distance

    ax.set_xlabel("Distance (light years)", fontsize=13)
    ax.set_ylabel("Time (years)", fontsize=13)
    ax.set_title(
        f"FTL Travel and Causality Violation\nBoost velocity: {v_current:.3f}c",
        fontsize=16,
        fontweight="bold",
    )
    ax.grid(True, alpha=0.3)
    ax.set_aspect("equal")
    ax.axhline(y=0, color="k", linewidth=0.5)
    ax.axvline(x=0, color="k", linewidth=0.5)

    # Draw Earth and Proxima worldlines
    ax.axvline(x=0, color="blue", linewidth=2.5, label="Earth", zorder=2)
    ax.axvline(x=distance, color="green", linewidth=2.5, label="Proxima", zorder=2)

    # Draw rest frame horizontal grid lines (faint)
    for t_val in np.arange(-5, 3, 0.5):
        ax.axhline(
            y=t_val, color="blue", linestyle="-", linewidth=0.5, alpha=0.15, zorder=0
        )

    # Draw one prominent rest frame simultaneity line
    ax.axhline(
        y=0,
        color="blue",
        linestyle="--",
        linewidth=2,
        label="Rest frame",
        alpha=0.5,
        zorder=1,
    )

    # Draw current boosted frame coordinate grid (animated) - VISIBLE GRID ONLY
    if v_current > 0.01:  # Only show when velocity is significant
        # First draw one line for the legend
        x_vals = np.array([0, 4.5])
        t_vals = 0 + np.tan(theta_current) * (x_vals - distance)
        ax.plot(
            x_vals,
            t_vals,
            color="orangered",
            linestyle="--",
            linewidth=1.2,
            alpha=0.5,
            zorder=0,
            label="Boosted frame",
        )

        # Draw the rest of the grid
        for t_offset in np.arange(-5, 3, 0.5):
            if abs(t_offset) > 0.01:  # Skip the one we already drew
                x_vals = np.array([0, 4.5])
                t_vals = t_offset + np.tan(theta_current) * (x_vals - distance)
                ax.plot(
                    x_vals,
                    t_vals,
                    color="orangered",
                    linestyle="--",
                    linewidth=1.2,
                    alpha=0.5,
                    zorder=0,
                )

    # Light cones at key events
    def draw_light_cone(x_event, t_event, extent=2.5):
        x_range = np.array([x_event - extent, x_event + extent])
        ax.plot(
            x_range,
            t_event + (x_range - x_event),
            "gray",
            linestyle=":",
            linewidth=1.5,
            alpha=0.5,
        )
        ax.plot(
            x_range,
            t_event - (x_range - x_event),
            "gray",
            linestyle=":",
            linewidth=1.5,
            alpha=0.5,
        )

    draw_light_cone(0, 0, extent=3.0)
    draw_light_cone(distance, 0, extent=3.0)
    if v_current > 0.01:
        draw_light_cone(0, time_shift_current, extent=2.5)

    # STAGE 1: Initial FTL jump to Proxima (PINK)
    ax.plot(
        [0, distance],
        [0, 0],
        "magenta",
        linewidth=3,
        label="FTL jumps",
        zorder=3,
        alpha=0.7,
    )
    ax.annotate(
        "",
        xy=(distance, 0),
        xytext=(0, 0),
        arrowprops=dict(arrowstyle="->", color="magenta", lw=3),
    )

    # STAGE 2: Traveler at Proxima (boost marker)
    ax.plot(
        [distance, distance],
        [-0.15, 0.15],
        "orange",
        linewidth=5,
        zorder=3,
        label=f"Boost to {v_current:.3f}c",
    )

    # STAGE 3: Return FTL jump (ORANGE - boosted frame) - no label
    if v_current > 0.01:
        ax.plot(
            [distance, 0],
            [0, time_shift_current],
            "darkorange",
            linewidth=3,
            zorder=2,  # Behind the boost marker
            alpha=0.7,
        )
        ax.annotate(
            "",
            xy=(0, time_shift_current),
            xytext=(distance, 0),
            arrowprops=dict(arrowstyle="->", color="darkorange", lw=3),
        )

    # Event markers
    ax.scatter(
        [0],
        [0],
        s=200,
        c="lime",
        marker="o",
        zorder=4,
        edgecolors="black",
        linewidths=2,
        label="Departure",
    )
    ax.scatter(
        [distance],
        [0],
        s=200,
        c="yellow",
        marker="o",
        zorder=4,
        edgecolors="black",
        linewidths=2,
    )

    if v_current > 0.01:
        ax.scatter(
            [0],
            [time_shift_current],
            s=250,
            c="red",
            marker="*",
            zorder=4,
            edgecolors="black",
            linewidths=2,
            label="Arrival (in past!)",
        )

        # Mark where boosted simultaneity intersects Earth
        ax.scatter(
            [0],
            [time_shift_current],
            s=180,
            c="purple",
            marker="s",
            zorder=3,
            edgecolors="black",
            linewidths=1,
            alpha=0.3,
        )

    # Annotations - position of arrival point
    if v_current > 0.01:
        ax.annotate(
            f"Earth 'now'\nat t={time_shift_current:.2f}",
            xy=(0, time_shift_current),
            xytext=(-1.8, time_shift_current - 0.8),
            fontsize=11,
            fontweight="bold",
            bbox=dict(boxstyle="round", facecolor="red", alpha=0.7),
            arrowprops=dict(arrowstyle="->", lw=2, color="red"),
        )

    # Explanatory text
    if v_current < 0.01:
        info_text = "Starting at rest frame (v=0)\nBoosting to 0.99c..."
    elif progress < 1.0:
        info_text = f"Boosting: {v_current:.3f}c\nSimultaneity rotating...\nTime shift: {time_shift_current:.2f} years"
    else:
        info_text = f"Final boost: {v_current:.3f}c\nReturn jump (orange) lands {abs(time_shift_current):.2f} years\nin the PAST!\n→ Causality violated!"

    ax.text(
        2.5,
        2.3,
        info_text,
        fontsize=11,
        ha="center",
        bbox=dict(boxstyle="round", facecolor="yellow", alpha=0.75, pad=0.8),
    )

    ax.set_xlim(-2.2, 5)
    ax.set_ylim(-5.5, 3)
    ax.legend(loc="upper left", fontsize=10, framealpha=0.9)

    return []


# Create animation
anim = FuncAnimation(
    fig, animate, init_func=init, frames=51, interval=50, blit=False, repeat=False
)

animation_created_time = time.perf_counter()

plt.close()  # Don't display the static figure

# Display animation using JavaScript (no ffmpeg required)
html_output = HTML(anim.to_jshtml())

end_time = time.perf_counter()

# Print benchmark results
print(f"Animation generation benchmark:")
print(f"  Setup + FuncAnimation creation: {animation_created_time - start_time:.3f} seconds")
print(f"  HTML/JavaScript conversion: {end_time - animation_created_time:.3f} seconds")
print(f"  Total time: {end_time - start_time:.3f} seconds")
print()

html_output